# Recreating Analysis of Piao Nature 2008

## Step 5: get tmp-co2 correlations and trends in ccgcrv outputs

In [ ]:
import pandas as pd
from piao2008_get_ccgcrv import get_ccgcrv
from piao2008_get_tmp_mean import get_tmp_mean
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# Prepare variables
co2_file = '../data/piao_2008/piao_co2_fm.csv'
stations_file = '../data/piao_2008/piao_stations.csv'
stations = pd.read_csv(stations_file)
calc_tmps = False

In [ ]:
# Get ccgcrv output
ccg_output, ccgmeans = get_ccgcrv(co2_file=co2_file, stations_file=stations_file, start_col='start_total', end_col='end_total')

In [ ]:
def get_station_tmp_mean(station):
    # Get temperature data
    S = int(stations.loc[stations['name']==station, 'lat']) - 20
    N = np.min([int(stations.loc[stations['name']==station, 'lat']) + 20, 90])
    tmp_mean = get_tmp_mean(tmp_file='/Users/moyanofe/BigData/GeoSpatial/Climate/CRU-TS_4.05_1901-2020/cru_ts4.05.1901.2020.tmp.dat.nc',
        limS=S,
        limN=N)
    file_out = 'tmp_yearly_means_' + station + '.csv'
    tmp_mean.to_csv('../data/piao_2008/' + file_out)

In [ ]:
if calc_tmps:
    for s in stations['name']:
        get_station_tmp_mean(s)

In [ ]:
# Function to make correlation and time series plots

def make_plots(station, start_year, end_year, detrend=False, name="", tx=0.5):
    # station: station abbreviation as given in file
    # start_year: earliest starting year to select either as integer or as column in file with starting years
    # end_year: same as start_year but for latest year to select
    # name: string to add in file names

    from scipy import stats

    # Prepare the data and calculate anomalies ----



    # Select time period for temperature data
    
    if isinstance(start_year, int):
        year_beg = start_year
    else:
        year_beg = int(stations.loc[stations['name']==station, start_year])

    if isinstance(end_year, int):
        year_end = end_year
    else:
        year_end = int(stations.loc[stations['name']==station, end_year])

    # Get the ccg data
    ccg_y = ccg_output[station]['yearly']
    azc = ccg_y['tcu_doy'].loc[ccg_y['year'].between(year_beg, year_end)]
    azc.loc[azc < 150] = azc.loc[azc < 150] + 365
    years = ccg_y['year'].loc[ccg_y['year'].between(year_beg, year_end)]

    # Get temperature data
    tmp_file = 'tmp_yearly_means_' + station + '.csv'
    tmp_mean = pd.read_csv('../data/piao_2008/' + tmp_file, index_col='year')
    tmp = tmp_mean.loc[years.iloc[0]:years.iloc[len(years)-1],'tmp_SON']

    # Calculate anomalies
    tmp_ano = tmp - tmp.mean()
    azc_ano = azc - azc.mean()

    xy = pd.DataFrame({'year': years, 'tmp_ano': tmp_ano.to_numpy(), 'azc_ano': azc_ano.to_numpy()})

    # Calculate regression and plot ----
    if detrend:
        lm = stats.linregress(xy['year'], xy['tmp_ano'])
        pred = lm.intercept + lm.slope * xy['year']
        xy.loc[:,'tmp_ano'] = xy['tmp_ano'] - pred

        lm = stats.linregress(xy['year'], xy['azc_ano'])
        pred = lm.intercept + lm.slope * xy['year']
        xy.loc[:,'azc_ano'] = xy['azc_ano'] - pred

    # Use scipy to get stats            
    # t, p = stats.kendalltau(x_in, y_in)[0] # used for ordinal data
    reg = stats.linregress(xy['tmp_ano'], xy['azc_ano'])
    r_val = "r = {:.3f}".format(reg.rvalue)
    p_val = "p = {:.3f}".format(reg.pvalue)
    azc_pred = reg.intercept + reg.slope * xy['tmp_ano']
    # r = np.sqrt(model.score(x, y))

    # Plot the correlation
    # xy = xy.sort_values(by='tmp_ano')
    plt.rcParams['figure.figsize'] = [5, 5]
    plt.plot(xy['tmp_ano'], xy['azc_ano'], 'o', color='grey', alpha=1, label='Temp anomaly vs AZC anomaly'+'\n'+r_val+'; '+p_val)
    plt.plot(xy['tmp_ano'], azc_pred, '--', color='grey')
    plt.title(label=station)
    plt.legend(loc='upper right')
    plt.show()

    # Plot the time series
    plt.rcParams['figure.figsize'] = [8, 5]
    fig,ax=plt.subplots()
    ax.plot(years, xy['azc_ano'], '--', color='red', label='AZC anomaly')
    ax.set_ylabel("Autumn zero crossing anomaly (days)", color="red")
    # ax.set_ylim([-15, 15])
    ax2=ax.twinx()
    ax2.plot(years, xy['tmp_ano'], '--', color='black', label='Autumn temperature anomaly', linewidth=2)
    ax2.set_ylabel("Autumn temperature anomaly (ºC)")
    # ax2.set_ylim([-2, 2])
    ax2.invert_yaxis()
    plt.title(station)
    plt.text(tx, 0.95, r_val+'; '+p_val, transform=ax.transAxes)
    plt.show()

    # Save the plot
    fig.savefig('../plots/' + name + '_' + station + '_azc-tmp.jpg',
                format='jpeg',
                dpi=100,
                bbox_inches='tight')   
    

In [ ]:
# Get results to reproduce Piao 2008 Figure 1a
start_col = 'start_piao2008'
end_col = 'end_piao2008'
name = "piao"
for s in stations['name']:
    make_piaofig1a(station=s, start_col=start_col, end_col=end_col, name=name)

### Testing specific cases

**Comparison of Point Barrow data**

- Short time series vs long time series
- Row anomalies vs detrended anomalies

In [ ]:
# Chose
station='BRW'
start = 'start_piao2008'
end = 'end_piao2008'
start = 1970
end = 1995
# start = 1996
# end = 2020
start = 1970
end = 2020
name = ''
make_plots(station=station, start_year=start, end_year=end, detrend=False, tx=0.5)
make_plots(station=station, start_year=start, end_year=end, detrend=True, tx=0.5)